In [43]:
import mysql.connector
from mysql.connector import Error

# Database Setup
class DatabaseSetup:
    def __init__(self, host="localhost", user="root", password="root"):
        self.host = host
        self.user = user
        self.password = password
        self.conn = None
        self.cursor = None

    def connect(self):
        try:
            self.conn = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password
            )
            if self.conn.is_connected():
                self.cursor = self.conn.cursor(dictionary=True)
                print("Connected to the MySQL server.")
        except Error as e:
            print(f"Error: {e}")

    def create_db(self, db_name):
        try:
            self.cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
            print(f"Database '{db_name}' is ready.")
        except Error as e:
            print(f"Error: {e}")

    def connect_to_db(self, db_name):
        try:
            self.conn.database = db_name
            print(f"Switched to database '{db_name}'.")
        except Error as e:
            print(f"Error: {e}")

    def close_connection(self):
        if self.conn.is_connected():
            self.cursor.close()
            self.conn.close()
            print("Connection to MySQL closed.")


# Customer Management
class Customer:
    def __init__(self, db):
        self.db = db

    def add_customer(self, name, email, phone):
        query = "INSERT INTO customers (name, email, phone) VALUES (%s, %s, %s)"
        try:
            self.db.cursor.execute(query, (name, email, phone))
            self.db.conn.commit()
            print("Customer added successfully.")
        except Error as e:
            print(f"Error adding customer: {e}")

    def view_customers(self):
        query = "SELECT * FROM customers WHERE status = 'Active'"
        try:
            self.db.cursor.execute(query)
            return self.db.cursor.fetchall()
        except Error as e:
            print(f"Error fetching customers: {e}")


# Product Management
class Product:
    def __init__(self, db):
        self.db = db

    def add_product(self, name, price, stock):
        query = "INSERT INTO products (name, price, stock) VALUES (%s, %s, %s)"
        try:
            self.db.cursor.execute(query, (name, price, stock))
            self.db.conn.commit()
            print("Product added successfully.")
        except Error as e:
            print(f"Error adding product: {e}")

    def view_products(self):
        query = "SELECT * FROM products WHERE status = 'Available'"
        try:
            self.db.cursor.execute(query)
            return self.db.cursor.fetchall()
        except Error as e:
            print(f"Error fetching products: {e}")


# Purchase Management
class Purchase:
    def __init__(self, db):
        self.db = db

    def add_purchase(self, customer_id, product_id, quantity):
        stock_query = "SELECT stock FROM products WHERE product_id = %s AND status = 'Available'"
        self.db.cursor.execute(stock_query, (product_id,))
        stock = self.db.cursor.fetchone()

        if stock and stock['stock'] >= quantity:
            purchase_query = '''
                INSERT INTO purchases (customer_id, product_id, quantity)
                VALUES (%s, %s, %s)
            '''
            update_stock_query = "UPDATE products SET stock = stock - %s WHERE product_id = %s"

            try:
                self.db.cursor.execute(purchase_query, (customer_id, product_id, quantity))
                self.db.cursor.execute(update_stock_query, (quantity, product_id))
                self.db.conn.commit()
                print("Purchase recorded successfully.")
            except Error as e:
                print(f"Error recording purchase: {e}")
        else:
            print("Insufficient stock or product unavailable.")

    def view_purchases(self):
        query = '''
            SELECT p.purchase_id, c.name AS customer, pr.name AS product, p.quantity, p.purchase_date
            FROM purchases p
            JOIN customers c ON p.customer_id = c.customer_id
            JOIN products pr ON p.product_id = pr.product_id
        '''
        try:
            self.db.cursor.execute(query)
            return self.db.cursor.fetchall()
        except Error as e:
            print(f"Error fetching purchases: {e}")


# Main Functionality
if __name__ == "__main__":
    db_setup = DatabaseSetup(host="localhost", user="root", password="root")
    db_setup.connect()

    # Create and connect to database
    db_name = "crpm"
    db_setup.create_db(db_name)
    db_setup.connect_to_db(db_name)

    # Initialize modules
    customer_ops = Customer(db_setup)
    product_ops = Product(db_setup)
    purchase_ops = Purchase(db_setup)

    # Test Customer Management
    customer_ops.add_customer("ronaldo", "cr7@gamil.com", "20574136")
    customer_ops.add_customer("kaka", "kaka@gmail.com", "8545367412")
    customer_ops.add_customer("pepe", "pepe@gmail.com", "987456253")
    print(customer_ops.view_customers())

    # Test Product Management
    product_ops.add_product("ipad", 25000, 10)
    product_ops.add_product("phone charger", 1500, 40)
    product_ops.add_product("headphones", 3000, 40)
    print(product_ops.view_products())

    # Test Purchase Management
    purchase_ops.add_purchase(1, 1, 2)
    print(purchase_ops.view_purchases())

    db_setup.close_connection()


Connected to the MySQL server.
Database 'crpm' is ready.
Switched to database 'crpm'.
Customer added successfully.
Customer added successfully.
Customer added successfully.
[{'customer_id': 1, 'name': 'John Doe', 'email': 'john@example.com', 'phone': '1234567890', 'status': 'Active'}, {'customer_id': 5, 'name': 'mira', 'email': 'mira@example.com', 'phone': '454547890', 'status': 'Active'}, {'customer_id': 6, 'name': 'ram', 'email': 'ram@example.com', 'phone': '6789012345', 'status': 'Active'}, {'customer_id': 13, 'name': 'karthi', 'email': 'dkarthi1@example.com', 'phone': '1234567891', 'status': 'Active'}, {'customer_id': 14, 'name': 'mira', 'email': 'mira1@example.com', 'phone': '454548585', 'status': 'Active'}, {'customer_id': 26, 'name': 'mira', 'email': 'mira_1@example.com', 'phone': '9003132458', 'status': 'Active'}, {'customer_id': 31, 'name': 'karthi', 'email': 'dkarthi_1@example.com', 'phone': '9003195742', 'status': 'Active'}, {'customer_id': 39, 'name': 'ram', 'email': 'ram1@